In [39]:
import torch
from torch import nn
import pandas as pd
from torch.utils.data import TensorDataset,Dataset,DataLoader,random_split

def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
    dataNorm["CreditLevel"]=dataset["CreditLevel"]-1
    return dataNorm

# data preprocessing
train_dataset = pd.read_csv("BankChurners.csv")
print(train_dataset.shape)
#train_dataset.head(10)
train_dataset = train_dataset.drop(['CustomerId','Geography'],axis=1)
train_dataset = normalize(train_dataset)
print(train_dataset.shape)
train_dataset.head()


(9000, 10)
(9000, 8)


,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,CreditLevel
0,0.3,0.484985,0.000000,1.0,0.0,0.643290,1.0,7
1,0.6,0.000000,0.000000,1.0,0.0,0.251062,1.0,6
2,0.2,0.728934,0.000000,1.0,0.0,0.015250,0.0,6
3,0.2,0.407651,0.333333,1.0,0.0,0.449146,0.0,1
4,0.7,0.435819,0.333333,1.0,0.0,0.513377,0.0,6


In [40]:
x_train = torch.tensor(train_dataset.drop('CreditLevel',axis=1).values.astype('float32'))
y_train = torch.tensor(train_dataset['CreditLevel'].values)
train_tensor = TensorDataset(x_train,y_train)
n_train = int(len(train_tensor)*0.8)
n_valid = len(train_tensor) - n_train
ds_train,ds_valid = random_split(train_tensor,[n_train,n_valid])
dl_train = DataLoader(ds_train,batch_size = 100,shuffle=True)
dl_valid_batch_size = 30
dl_valid = DataLoader(ds_valid,batch_size = dl_valid_batch_size,shuffle=False)

for features,labels in dl_train:
    print(features,labels)
    break

tensor([[0.9000, 0.4241, 0.3333, 0.0000, 0.0000, 0.3875, 0.0000],
        [0.3000, 0.0000, 0.3333, 1.0000, 0.0000, 0.8590, 0.0000],
        [0.1000, 0.5316, 0.0000, 1.0000, 0.0000, 0.5120, 1.0000],
        [0.3000, 0.6060, 0.0000, 1.0000, 1.0000, 0.2672, 1.0000],
        [0.2000, 0.8361, 0.3333, 1.0000, 1.0000, 0.7536, 1.0000],
        [0.9000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5005, 0.0000],
        [0.3000, 0.5489, 0.0000, 0.0000, 1.0000, 0.7971, 0.0000],
        [0.1000, 0.7195, 0.0000, 0.0000, 0.0000, 0.6173, 0.0000],
        [0.3000, 0.4941, 0.3333, 1.0000, 0.0000, 0.7391, 0.0000],
        [0.8000, 0.6401, 0.3333, 1.0000, 0.0000, 0.6464, 0.0000],
        [0.8000, 0.5723, 0.0000, 1.0000, 1.0000, 0.1126, 0.0000],
        [0.3000, 0.5809, 0.0000, 1.0000, 1.0000, 0.5073, 0.0000],
        [1.0000, 0.0000, 0.3333, 1.0000, 1.0000, 0.7993, 1.0000],
        [0.3000, 0.0000, 0.0000, 1.0000, 0.0000, 0.4740, 0.0000],
        [0.0000, 0.3909, 0.0000, 1.0000, 0.0000, 0.2621, 0.0000],
        [0

In [41]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import kaiming_uniform_,xavier_uniform_

class Net(nn.Module):
    def __init__(self,n_inputs):
        super(Net, self).__init__()
        # input to first hidden layer
        self.hidden1 = nn.Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.dropout1 = nn.Dropout(0.3)
        self.act1 = nn.ReLU()
        # second hidden layer
        self.hidden2 = nn.Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        #self.dropout2 = nn.Dropout(0.3)
        self.act2 = nn.ReLU()
        # third hidden layer and output
        self.hidden3 = nn.Linear(8, 10)
        xavier_uniform_(self.hidden3.weight)
        #self.dropout3 = nn.Dropout(0.3)
        self.act3 = nn.Softmax(dim=1)
    
    def forward(self,X):
         # input to first hidden layer
        X = self.hidden1(X)
        X = self.dropout1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        #X = self.dropout2(X)
        X = self.act2(X)
        # output layer
        X = self.hidden3(X)
        #X = self.dropout3(X)
        X = self.act3(X)
        return X

model = Net(7)
print(model)

Net(
  (hidden1): Linear(in_features=7, out_features=10, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (act1): ReLU()
  (hidden2): Linear(in_features=10, out_features=8, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=8, out_features=10, bias=True)
  (act3): Softmax(dim=1)
)


In [42]:
#define a Loss function and optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [43]:
# Train the network
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs,targets) in enumerate(dl_train):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,     1] loss: 0.023
[2,     1] loss: 0.023
[3,     1] loss: 0.023
[4,     1] loss: 0.023
[5,     1] loss: 0.023
[6,     1] loss: 0.023
[7,     1] loss: 0.022
[8,     1] loss: 0.023
[9,     1] loss: 0.023
[10,     1] loss: 0.022
[11,     1] loss: 0.023
[12,     1] loss: 0.022
[13,     1] loss: 0.023
[14,     1] loss: 0.023
[15,     1] loss: 0.022
[16,     1] loss: 0.023
[17,     1] loss: 0.023
[18,     1] loss: 0.023
[19,     1] loss: 0.023
[20,     1] loss: 0.022


In [44]:
# model test
model.eval()
accuracy = 0.0
total = 0.0
with torch.no_grad():
    for data in dl_valid:
        inputs,targets = data
        outputs = model(inputs)
        _,predicted = torch.max(outputs.data,1)
        total += targets.size(0)
        accuracy += (predicted == targets).sum().item()

accuracy = (100*accuracy/total)
print(accuracy)

21.27777777777778


In [49]:
def testClassess():
    number_of_labels = 10
    class_correct = list(0. for i in range(number_of_labels))
    class_total = list(0. for i in range(number_of_labels))
    with torch.no_grad():
        for data in dl_valid:
            inputs,targets = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == targets).squeeze()
            for i in range(dl_valid_batch_size):
                target = targets[i]
                class_correct[target] += c[i].item()
                class_total[target] += 1

    for i in range(number_of_labels):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))
        
        
testClassess()

NameError: name 'classes' is not defined